In [69]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os

In [70]:
query_path = r"C:\Users\raphael.almeida\Documents\Projetos\Boletos Reemitidos\sql\reissued_invoices.sql"
with open (query_path, 'r') as file:
    query=file.read()

df_reemitidos = awr.athena.read_sql_query(query,database='silver')
df_reemitidos.drop_duplicates(subset='ponteiro', inplace=True)

gerando lista e acrescentando colunas

In [71]:
lista_reemitidos = df_reemitidos['ponteiro'].to_list()

df_reemitidos['data_baixa'] = pd.NA
df_reemitidos['valor_baixa'] = pd.NA
df_reemitidos['situacao'] = 'REEMITIDO'

In [72]:
query_path = r"C:\Users\raphael.almeida\Documents\Projetos\Boletos Reemitidos\sql\paid_invoices.sql"
with open (query_path, 'r') as file:
    query=file.read()

df_pagos = awr.athena.read_sql_query(query,database='silver')
df_pagos.drop_duplicates(subset='ponteiro', inplace=True)

acrescentando colunas e filtrando por ponteiros reemitidos

In [73]:
df_pagos['data_reemissao'] = pd.NA
df_pagos['situacao'] = 'PAGO'
df_reemitidos_pagos = df_pagos[df_pagos['ponteiro'].isin(lista_reemitidos)]

concatenando

In [74]:
df_final = pd.concat([df_reemitidos_pagos, df_reemitidos])

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_28876\4118452813.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_reemitidos_pagos, df_reemitidos])


reorganizando colunas

In [75]:

colunas_ordenadas = [
    'codigo_cadastro',
    'ponteiro',
    'numero_documento',
    'numero_boleto',
    'nosso_numero',
    'sequencia_documento',
    'aplicacao_financeira',
    'valor_titulo',
    'valor_baixa',
    'situacao',
    'data_emissao',
    'data_reemissao',
    'data_baixa',
    'data_vencimento',
    'conjunto',
    'matricula',
    'unidade',
    'empresa',
    'associado',
    'vendedor',
    'grupo'
]

df_final = df_final[colunas_ordenadas]

In [76]:
df_final.columns

Index(['codigo_cadastro', 'ponteiro', 'numero_documento', 'numero_boleto',
       'nosso_numero', 'sequencia_documento', 'aplicacao_financeira',
       'valor_titulo', 'valor_baixa', 'situacao', 'data_emissao',
       'data_reemissao', 'data_baixa', 'data_vencimento', 'conjunto',
       'matricula', 'unidade', 'empresa', 'associado', 'vendedor', 'grupo'],
      dtype='object')

tratando nulos

In [77]:
df_final = df_final.sort_values(by=['ponteiro','data_reemissao'], ascending=False)

df_final['conjunto'] = df_final['conjunto'].astype(str)
df_final['matricula'] = df_final['matricula'].astype(str)
df_final['unidade'] = df_final['unidade'].astype(str)
df_final['numero_boleto'] = df_final['numero_boleto'].astype(str)
df_final['nosso_numero'] = df_final['nosso_numero'].astype(str)


df_final['valor_baixa'] = df_final['valor_baixa'].fillna(0)
df_final['data_baixa'] = df_final['data_baixa'].fillna(pd.Timestamp('1900-01-01').date())
df_final['conjunto'] = df_final['conjunto'].fillna('NULL')
df_final['matricula'] = df_final['matricula'].fillna('NULL')
df_final['unidade'] = df_final['unidade'].fillna('NULL')
df_final['data_reemissao'] = df_final['data_reemissao'].fillna(pd.Timestamp('1900-01-01').date())
df_final['numero_boleto'] = df_final['numero_boleto'].fillna('NULL')
df_final['nosso_numero'] = df_final['nosso_numero'].fillna('NULL')



passando pro excel

In [78]:
today = pd.Timestamp.today().date()
today_format = today.strftime('%Y%m%d')

In [79]:
save_path_new = r"C:\Users\raphael.almeida\Documents\Projetos\Boletos Reemitidos"
save_path_cache = r"C:\Users\raphael.almeida\Documents\Projetos\Boletos Reemitidos\cache"

arquivo_new = os.path.join(save_path_new, 'boletos_reemitidos.xlsx')
arquivo_cache = os.path.join(save_path_cache, f'boletos_reemitidos_{today_format}.xlsx')

df_final.to_excel(arquivo_new, engine='openpyxl', index=False, sheet_name='boletos_reemitidos_pagos')
df_final.to_excel(arquivo_cache, engine='openpyxl', index=False, sheet_name='boletos_reemitidos_pagos')

passando para o excel de visualização

In [84]:
!pip3 install xlwings



In [85]:
import xlwings


ModuleNotFoundError: No module named 'xlwings'